In [1]:
import pandas as pd
import requests

In [2]:
text_file = open("2022purchsedISBN.txt", "r")
lines = text_file.readlines()
purlist = [i.strip("\n") for i in lines]

In [51]:
portlist = pd.read_csv("Elsevier+eBooks+(EBA+Purchased)_portfolios-csv.csv", encoding = "utf-8")

In [66]:
portlist

,LOCALIZED,ISSN,ISSN.1,ISSN.2,ISBN,ISBN.1,ISBN.2,OTHER_SYSTEM_NUMBER,OTHER_SYSTEM_NUMBER.1,OTHER_SYSTEM_NUMBER.2,...,LICENSE_NAME,COLLECTION_LICENSE,COLLECTION_LICENSE_NAME,PDA,EBA,PO_LINE,LANGUAGE,COUNTER_PLATFORM,NOTES,MarkBuy
0,N,NaN,NaN,NaN,0128042710,0128134224,9780128134221,(OCoLC)on1090396924,(OCoLC)1090396924,(OCoLC)1229464007,...,NaN,NaN,NaN,NaN,NaN,POL-12982,eng,NaN,NaN,
1,N,NaN,NaN,NaN,0128171030,0128171049,9780128171042,(OCoLC)on1089449187,(OCoLC)1089449187,(OCoLC)1296541005,...,NaN,NaN,NaN,NaN,NaN,POL-12982,eng,NaN,NaN,
2,N,NaN,NaN,NaN,9780128167984,0128167998,9780128167991,(OCoLC)on1076485471,(OCoLC)1076485471,(OCoLC)1076793874,...,NaN,NaN,NaN,NaN,NaN,POL-12982,eng,NaN,NaN,
3,N,NaN,NaN,NaN,9780128133668,0128133678,9780128133675,(OCoLC)on1081422793,(OCoLC)1081422793,(OCoLC)1082307968,...,NaN,NaN,NaN,NaN,NaN,POL-12982,eng,NaN,NaN,
4,N,NaN,NaN,NaN,9780128149485,0128149485,9780128149485,(OCoLC)on1051340762,(OCoLC)1051340762,(OCoLC)1151672720,...,NaN,NaN,NaN,NaN,NaN,POL-12982,eng,NaN,NaN,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3588,N,NaN,NaN,NaN,9780128194898,0323989241,9780323989244,(OCoLC)on1261765001,(OCoLC)1261765001,NaN,...,NaN,NaN,NaN,NaN,NaN,POL-12982,eng,NaN,NaN,
3589,N,NaN,NaN,NaN,9780128222126,012821757X,9780128217573,(OCoLC)on1285337063,(OCoLC)1285337063,(OCoLC)1285487422,...,NaN,NaN,NaN,NaN,NaN,POL-12982,eng,NaN,NaN,
3590,N,NaN,NaN,NaN,9780128194874,NaN,NaN,(OCoLC)on1247471188,(OCoLC)1247471188,NaN,...,NaN,NaN,NaN,NaN,NaN,POL-12982,eng,NaN,NaN,
3591,N,NaN,NaN,NaN,9780323898126,0323898130,9780323898133,(OCoLC)on1203050129,(OCoLC)1203050129,NaN,...,NaN,NaN,NaN,NaN,NaN,POL-12982,eng,NaN,NaN,


In [52]:
portlist["MarkBuy"] = ""

In [53]:
incollection = []

In [54]:
for i, row in portlist.iterrows():
    newurl = row["URL"].lstrip("http://libproxy.sdsu.edu/login?url=https://www.sciencedirect.com/science/book/")
    isbnr = [str(row["ISBN"]), str(row["ISBN.1"]), str(row["ISBN.2"]), newurl]
    for n in purlist:
        if n in isbnr:
            incollection.append(n)
            portlist.at[i,'MarkBuy'] = "Y"

In [55]:
for i in purlist:
    if i not in incollection:
        print(i)

9780081020791


In [59]:
portfolio_list = portlist[portlist["MarkBuy"]=="Y"].PORTFOLIO_PID.to_list()

In [35]:
portlist.to_csv("afteraddingbuynote.csv")

In [39]:
collection_id = "61423368350002917"
service_id="62442316160002917"

In [41]:
baseuri = "https://api-na.hosted.exlibrisgroup.com/almaws/v1/electronic/e-collections/"+collection_id+"/e-services/"+service_id+"/portfolios/"
apikey = "?apikey=fake"

for i in portfolio_list:
    testpid = i
    getruri = baseuri + testpid + apikey
    rg1 = requests.get(getruri, headers = {'Accept': 'application/json'})
    base1 = rg1.json()
    datat1 = {}
    data1["access_type"]={"value": "perpetual","desc": "Perpetual"}
    datat1["material_type"] = base1["material_type"]
    datat1['coverage_details']= base1["coverage_details"] 
    datat1["public_note"] = base1["public_note"]
    datat1['linking_details'] = base1['linking_details']
    datat1["library"] = base1["library"]
    datat1['activation_date']=base1['activation_date']
    datat1['id'] = testpid
    ruri = baseuri + testpid + apikey
    r1 = requests.put(ruri, json=datat1, headers={"Content-Type": "application/json"})
    print(testpid, r1)

17